## Import Libs

In [1]:
import os
import warnings # 避免一些可以忽略的报错
warnings.filterwarnings('ignore')
import random
import gc
import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm # 进度条
import time

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

## CONFIG

In [2]:
is_debug = False

class CONFIG:
    seed = 308

    test_batch_size = 512
    
    in_features = 784
    n_classes = 10

    n_workers = 1
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    test_csv = "/kaggle/input/digit-recognizer/test.csv"
    ckpt_path = "/kaggle/input/digit-recognizer-infer-3mlp/CV_0.9830_Loss1.4865_epoch8.bin"

## Set Random Seed

In [3]:
def set_seed(seed=308):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
set_seed(CONFIG.seed) # 固定随机种子，方便结果复现

## Data Progress

In [4]:
test = pd.read_csv(CONFIG.test_csv) # 读取 测试集 的数据
test

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Dataset and DataLoader

In [5]:
class MyDataset(Dataset):
    def __init__(self, df):
        super().__init__()
        self.df = df
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx, :] # 从 df 中取出 idx 这一行
        X = row["pixel0":].values
        
        return X, str(idx) # 测试集的数据没有 label，我们用它对应的索引代替

In [6]:
def prepare_loaders():
    test_datasets = MyDataset(df=test)
    
    test_loader = DataLoader(test_datasets, batch_size=CONFIG.test_batch_size, num_workers=CONFIG.n_workers, shuffle=False, pin_memory=True)
    # 推理过程对数据按顺序执行，不进行打乱操作，shuffle为False
    
    return test_loader

## Model

In [7]:
# 模型的定义同训练，模型相同才能加载训练的 notebook 中训练得到的模型权重
def MLP_Layer(in_features, out_features):
    linear = nn.Linear(in_features=in_features, out_features=out_features)
    norm = nn.BatchNorm1d(out_features)
    act = nn.LeakyReLU(0.01)
    
    return nn.Sequential(
        linear,
        norm,
        act
    )

In [8]:
# 模型的定义同训练，模型相同才能加载训练的 notebook 中训练得到的模型权重
class DigitRecognizerModel(nn.Module):
    def __init__(self):
        super(DigitRecognizerModel, self).__init__()
        self.model = nn.Sequential(
            MLP_Layer(in_features=CONFIG.in_features, out_features=1024),
            MLP_Layer(in_features=1024, out_features=384),
            MLP_Layer(in_features=384, out_features=CONFIG.n_classes)
        )
        
    def forward(self, x):
        output = self.model(x)
        return output

## Load Model

In [9]:
model = DigitRecognizerModel() # 实例化模型
model.to(CONFIG.device)

# 加载训练得到的模型的权重参数
model.load_state_dict(torch.load(CONFIG.ckpt_path, map_location=CONFIG.device)) # map_location作用：防止不同device之间的模型与参数无法加载

<All keys matched successfully>

## Infer Function

In [10]:
def Infer(model, test_loader):
    model.eval()
    
    y_preds = []
    bar = tqdm(enumerate(test_loader), total=len(test_loader))
    with torch.no_grad():
        for step, (X, X_id) in bar:

            X = X.to(CONFIG.device, dtype=torch.float)

            outputs = model(X)
            outputs = F.softmax(outputs)

            y_preds.append(outputs.argmax(1).detach().cpu().numpy())
            
    y_preds = np.concatenate(y_preds)

    return y_preds

## Start Infer

In [11]:
test_loader = prepare_loaders()

In [12]:
y_preds = Infer(model, test_loader)

100%|██████████| 55/55 [00:03<00:00, 14.34it/s]


## Make Submission

In [13]:
sub = pd.DataFrame()
sub["ImageId"] = test.index + 1
sub["Label"] = y_preds
sub

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


In [14]:
sub.to_csv('submission.csv', index=False)
pd.read_csv('submission.csv')

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9
